In [1]:
import numpy as np

from manim import *

Manim Community v0.10.0

In [2]:
config.media_width = "60%"

# The Plateau Function

The plateau function is defined using the following formula.
It creates a smooth box-like function with its value is always between 0 and 1.
The plateau function can be seen as a multiplication of two sigmoids, one of which is flipped horizontally.

This plateau function has 3 parameters.

The center location of the plateau function is determined by the lambda_c parameter.

The width of the platea function is determined by the lambda_w paramter. The Plateau function has high values in a region of 2 times lambda_w.

and the slope of the platea function is determined by the lambda_s parameter. 
The slope of the platea function at the middle of the transition part on the left side
is equal to lambda_s devided by 4.


In [4]:
%%manim -v WARNING SinlgePlateau
# %%manim -v WARNING -qm SinlgePlateau

def plateau(t: float, c: float, w: float, s: float) -> float:
    inside_1 = s * (t - c - w)
    inside_2 = s * (-t + c - w)
    
    return 1. / ((np.exp(inside_1) + 1) * (np.exp(inside_2) + 1))


def plateau_1(t: float, c: float, w: float, s: float) -> float:
    inside_1 = s * (t - c - w)
    
    return 1. / ((np.exp(inside_1) + 1))


def plateau_2(t: float, c: float, w: float, s: float) -> float:
    inside_2 = s * (-t + c - w)
    
    return 1. / ((np.exp(inside_2) + 1))


class SinlgePlateau(Scene):
    def construct(self):
        lambda_c = ValueTracker(10.0)
        lambda_c_tex = r"\lambda^c"
        lambda_w = ValueTracker(4.0)
        lambda_w_tex = r"\lambda^w"
        lambda_s = ValueTracker(5.0)
        lambda_s_tex = r"\lambda^s"
        
        text_1 = Tex("The Plateau function", tex_to_color_map={"Plateau": BLUE})
        plateau_formula = MathTex(
            r"f(t|\lambda^c, \lambda^w, \lambda^s) = ",
            r"\frac{1}{(e^{\lambda^s(t-\lambda^c-\lambda^w)} + 1)",
            r"(e^{\lambda^s(-t+\lambda^c-\lambda^w)} + 1)}",
            substrings_to_isolate=[lambda_c_tex, lambda_w_tex, lambda_s_tex]
        )
        
        plateau_formula_seperate = MathTex(
            r"f(t|\lambda^c, \lambda^w, \lambda^s) = ",
            r"\frac{1}{e^{\lambda^s(t-\lambda^c-\lambda^w)} + 1}",
            r"~",
            r"\frac{1}{e^{\lambda^s(-t+\lambda^c-\lambda^w)} + 1}",
        )
        
        group = VGroup(text_1, plateau_formula)
        group.arrange(DOWN)
        
        self.play(Write(text_1))
        self.play(Write(plateau_formula))
        self.wait()
        
        self.play(group.animate.to_corner(UP).scale(1.0))
        plateau_formula_seperate.scale(1.0).move_to(plateau_formula)
        
        
        axes = Axes(
            x_range=[-0.9, 21.3, 2],
            y_range=[-0.15, 1.3, 1],
            x_axis_config={"numbers_to_exclude": [21]},
            y_axis_config={"numbers_to_include": [0, 1]},
            x_length=10,
            y_length=5,
            axis_config={"color": WHITE},
        )
        
        y_label = axes.get_y_axis_label("f")
        x_label = axes.get_x_axis_label("t")
        plot_labels = VGroup(x_label, y_label)
        
        
        plateau_graph = always_redraw(lambda: axes.get_graph(
            lambda x: plateau(x, lambda_c.get_value(), lambda_w.get_value(), lambda_s.get_value()),
            color=BLUE,
            stroke_width=8,
            x_range=[0, 20]
        ))
        
        plateau_graph_1 = axes.get_graph(
            lambda x: plateau_1(x, lambda_c.get_value(), lambda_w.get_value(), lambda_s.get_value()),
            color=GREEN,
            stroke_width=5,
            x_range=[0, 20]
        )
        
        plateau_graph_2 = axes.get_graph(
            lambda x: plateau_2(x, lambda_c.get_value(), lambda_w.get_value(), lambda_s.get_value()),
            color=GOLD,
            stroke_width=5,
            x_range=[0, 20]
        )
        
        the_graph = VGroup(axes, plateau_graph, plateau_graph_1, plateau_graph_2, plot_labels)
        the_graph.next_to(group, DOWN)
        
        self.play(Create(axes), Create(plot_labels))
        self.play(Create(plateau_graph), run_time=2)
        self.wait()
        
        ########################################################
        # the two parts of 
        ########################################################
        self.play(TransformMatchingTex(plateau_formula, plateau_formula_seperate))
        self.wait()
        box1 = SurroundingRectangle(plateau_formula_seperate[1], buff=0.1, color=GREEN)
        box2 = SurroundingRectangle(plateau_formula_seperate[3], buff=0.1, color=GOLD)
        self.play(AnimationGroup(
            Create(box1),
            Create(plateau_graph_1), lag_ratio=2
        ), run_time=2)
        self.wait()
        self.play(AnimationGroup(
            Create(box2),
            Create(plateau_graph_2), lag_ratio=2
        ), run_time=2)
        self.wait()
        
        self.remove(box1, plateau_graph_1, box2, plateau_graph_2)
        self.play(TransformMatchingTex(plateau_formula_seperate, plateau_formula))
        self.wait(3)
        
        ########################################################
        # animating lambda_c
        ########################################################
        lambda_c_highlighted = plateau_formula.set_color_by_tex(lambda_c_tex, RED)
        # the lambda_c arrow
        lambda_c_annot = Vector(DOWN).add_updater(
            lambda m: m.next_to(axes.c2p(lambda_c.get_value()), UP)
        )
        lambda_c_annot_label = MathTex(r"\lambda^c", color=RED).add_updater(lambda m: m.next_to(lambda_c_annot, UP))
        
        # add
        self.play(Transform(plateau_formula, lambda_c_highlighted))
        self.add(lambda_c_annot, lambda_c_annot_label)
        self.wait()
        
        # animation
        self.play(lambda_c.animate.increment_value(-4))
        self.wait(0.5)
        self.play(lambda_c.animate.increment_value(+8))
        self.wait(0.5)
        self.play(lambda_c.animate.increment_value(-4))
        self.wait(1.0)
        
        # clean up
        plateau_formula.set_color_by_tex(lambda_c_tex, WHITE)
        self.remove(lambda_c_annot, lambda_c_annot_label)
        self.play(Transform(lambda_c_highlighted, plateau_formula))
        
        ########################################################
        # animating lambda_w
        ########################################################
        lambda_w_highlighted = plateau_formula.set_color_by_tex(lambda_w_tex, RED)
        # the lambda_w arrow
        lambda_w_annot = DoubleArrow(
            start=axes.c2p(lambda_c.get_value() - lambda_w.get_value()),
            end=axes.c2p(lambda_c.get_value() + lambda_w.get_value())
        ).shift(UP * 0.75).add_updater(
            lambda m: m.become(DoubleArrow(
                start=axes.c2p(lambda_c.get_value() - lambda_w.get_value()),
                end=axes.c2p(lambda_c.get_value() + lambda_w.get_value())
            ).shift(UP * 0.75))
        )
        lambda_w_annot_label = MathTex(r"2 \lambda^w", color=RED).next_to(lambda_w_annot, UP)
        
        # add
        self.play(Transform(plateau_formula, lambda_w_highlighted))
        self.add(lambda_w_annot, lambda_w_annot_label)
        self.wait()
        
        # animation
        self.play(lambda_w.animate.increment_value(+4))
        self.wait()
        self.play(lambda_w.animate.increment_value(-4))
        self.wait()
        
        # cleanup
        plateau_formula.set_color_by_tex(lambda_w_tex, WHITE)
        self.remove(lambda_w_annot, lambda_w_annot_label)
        self.play(Transform(lambda_w_highlighted, plateau_formula))
        
        ########################################################
        # animating lambda_s
        ########################################################
        lambda_s_highlighted = plateau_formula.set_color_by_tex(lambda_s_tex, RED)
        # the lambda_s annotation
        lambda_s_line = TangentLine(
            plateau_graph,
            alpha=(lambda_c.get_value() - lambda_w.get_value()) / 20,
            length=3,
            color=WHITE
        ).add_updater(
            lambda m: m.become(
                TangentLine(
                    plateau_graph,
                    alpha=(lambda_c.get_value() - lambda_w.get_value()) / 20,
                    length=3,
                    color=WHITE
                )
            )
        )
        lambda_s_annot_point = Dot(axes.input_to_graph_point(lambda_c.get_value() - lambda_w.get_value(), plateau_graph))
        lambda_s_annot_tex = MathTex(r"\frac{\lambda^s}{4}", color=RED).next_to(lambda_s_annot_point, LEFT)
        
        
        # add
        self.play(Transform(plateau_formula, lambda_s_highlighted))
        self.add(lambda_s_line, lambda_s_annot_point, lambda_s_annot_tex)
        
        # animation
        self.play(lambda_s.animate.increment_value(+4), run_time=1)
        self.wait()
        self.play(lambda_s.animate.increment_value(-4.25-4), rate_func=exponential_decay, run_time=3)
        self.play(lambda_s.animate.increment_value(+4.25), run_time=1)
        self.wait()
        
        
        # cleanup
        plateau_formula.set_color_by_tex(lambda_s_tex, WHITE)
        self.remove(lambda_s_line, lambda_s_annot_point, lambda_s_annot_tex)
        self.play(Transform(lambda_s_highlighted, plateau_formula))
        
        
        self.wait(3)